In [18]:
import json 

from os import listdir
from os.path import isfile, join

import pandas as pd
import numpy as np 
%matplotlib inline
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 


class Result(object):
    
    def __init__(self, file_path):
        temp = file_path.split('_')
        self.symbols = int(temp[6])
        self.symbol_size = int(temp[7])
        self.results = read_result(file_path)      
        
    def data_size(self, unit=None):
        
        if unit == None:
            return self.symbols * self.symbol_size
        
    def less(self, other):
        if self.data_size() < other.data_size():
            return True
        elif self.data_size() == other.data_size():
            return self.symbols < other.symbols
        else:
            return False
        
    def avg_exec_time(self, unit=None):
        total = 0.0
        for res in self.results: 
            total = total + (res['end'] - res['start'])
        total = total / len(self.results)
        
        if not unit: 
            return total
        if unit == 'ms':
            return total / 1000
        
        if unit == 'sec':
            return total / 1000000
        
    @staticmethod
    def read_result(path):
        with open(path) as f:
            data = json.load(f)
            return data
        
def print_results(results):
    
    for res in results: 
        print('symbols: {} symbol size: {} data size: {}'.format(res.symbols, res.symbol_size, res.data_size()))

        
                
def is_sorted(res):
    for i in range(0, len(res) - 1):
        if not res[i].less(res[i + 1]):
            return False
    return True
    
def sort_results(res):
    for i in range(0, len(res) - 1):
        if not res[i].less(res[i + 1]):
            temp = res[i]
            res[i] = res[i + 1]
            res[i + 1] = temp
    if is_sorted(res):
        return res
    else:
        return sort_results(res)

def read_result(path):
    with open(path) as f:
        data = json.load(f)
        return data
    
def get_exec_time(obj):
    return obj['end'] - obj['start']

def micro_to_sec(val):
    return val / 1000000

def get_avg_exec_time(data, unit=None):
    total = 0
    for obj in data: 
        total = total + get_exec_time(obj)
    res = total / len(data)
    if not unit:
        return res
    elif unit is 'sec':
        return micro_to_sec(res)
    
def plot_bar(res):
    
    N = len(res)
    
    one_mb = [r for r in res if r.data_size() == 1048576]
    teen_mb = [r for r in res if r.data_size() == 10485760 if r.data_size() == 10485760 ]
    
    print('x')
    print_results(one_mb)
    
    fig, ax = plt.subplots()
    
    ind = np.arrange(N)
    width = 0.35
    
    p1 = ax.bar(ind, one_mb, color='r')
    
    
    
    
path = '../results/1536148910_single_thread_block_coder_8_16_65536'

path = '../results/'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

results = list()
for file in onlyfiles:
    temp = path + file
    result = Result(temp)
    results.append(result)
    
results = np.array(results)



a = [r.symbols for r in results]
b = np.argsort(a)
results = results[b]

a = [r.data_size() for r in results]
b = np.argsort(a, kind='mergesort')
results = results[b]


print_results(results)

plot_bar(results)

for res in results: 
    print(res.avg_exec_time('ms'))



#print(get_avg_exec_time(a.results, 'sec'))

    

symbols: 8 symbol size: 131072 data size: 1048576
symbols: 16 symbol size: 65536 data size: 1048576
symbols: 32 symbol size: 32768 data size: 1048576
symbols: 8 symbol size: 1310720 data size: 10485760
symbols: 16 symbol size: 655360 data size: 10485760
symbols: 32 symbol size: 327680 data size: 10485760
symbols: 8 symbol size: 13107200 data size: 104857600
symbols: 16 symbol size: 6553600 data size: 104857600
symbols: 32 symbol size: 3276800 data size: 104857600
symbols: 8 symbol size: 16777216 data size: 134217728
symbols: 16 symbol size: 8388608 data size: 134217728
symbols: 32 symbol size: 4194304 data size: 134217728
symbols: 8 symbol size: 33554432 data size: 268435456
symbols: 16 symbol size: 16777216 data size: 268435456
symbols: 32 symbol size: 8388608 data size: 268435456
symbols: 8 symbol size: 67108864 data size: 536870912
symbols: 16 symbol size: 33554432 data size: 536870912
symbols: 32 symbol size: 16777216 data size: 536870912
symbols: 8 symbol size: 134217728 data size